In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from imp import reload
from program import greedy
from program import classify
from program import input_data
from program import metrics as me
from copy import deepcopy as dcp

C:\Users\hasee\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Users\hasee\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# 全局设置

In [2]:
global_seed = 0
model = [greedy,classify,input_data,me]
for i in model:
    i.gloabl_seed = global_seed
del model,global_seed

# 开关

In [3]:
switch = [
    0,  0,     0,        0,    1,            1]

划分数据 Smote Normalize,Greedy,Greedy-metrics,All-metrics

# 分类器开关

In [30]:
DT=classify.DT
ANN=classify.ANN
Adaboost=classify.Adaboost
Xgboost=classify.Xgboost
Gboosting=classify.Gboosting
NB=classify.NB
VFDT=classify.VFDT

clf =      [DT,ANN,Adaboost,Xgboost,Gboosting,NB,VFDT]
clf_name = ['DT','ANN','Adaboost','Xgboost','Gboosting','NB','VFDT']
clf_switch=[0,   0,     0,         0,        0,          0,   1    ]

# 划分数据

In [5]:
save_path = r'result/data_after_split.npy'

if switch[0]:
    data,title = input_data.read_data()

    #变量顺序为 xtrain,xtest,ytrain,ytest
    data_after_split = np.empty(4,dtype = pd.core.frame.DataFrame)
    data_after_split[0],data_after_split[1],data_after_split[2],data_after_split[3] = input_data.data_split(data)

    np.save(save_path,data_after_split)
    print('columns of data :')
    print(title)
    print('data has saved in ' + save_path)
    del data,title,data_after_split

In [6]:
print('load data in '+ save_path)
data_after_split = np.load(save_path,allow_pickle = True)
xtrain,xtest,ytrain,ytest = data_after_split[0],data_after_split[1],data_after_split[2],data_after_split[3]
del data_after_split

load data in result/data_after_split.npy


# Smote

In [7]:
save_path = r'result/data_after_smote.npy'
if switch[1]:
    xtrain_s1,ytrain_s1,xtrain_s2,ytrain_s2 = input_data.smote(xtrain,ytrain)
    data_after_smote = np.empty(4,dtype = pd.core.frame.DataFrame)
    data_after_smote[0],data_after_smote[1],data_after_smote[2],data_after_smote[3] = xtrain_s1,ytrain_s1,xtrain_s2,ytrain_s2

    np.save(save_path,data_after_smote)
    print('data has saved in ' + save_path)
    del xtrain_s1,ytrain_s1,xtrain_s2,ytrain_s2,data_after_smote

In [8]:
data_after_smote = np.load(r'result/data_after_smote.npy',allow_pickle=True)
xtrain_s1,ytrain_s1,xtrain_s2,ytrain_s2 = data_after_smote[0],data_after_smote[1],data_after_smote[2],data_after_smote[3]
del data_after_smote

# Normalize

In [9]:
save_path = r'result/data_norm.npz'
if switch[2]:
    xtrain_norm = input_data.normalize(xtrain,xtrain)
    xtrain_s1_norm = input_data.normalize(xtrain_s1,xtrain_s1)
    xtrain_s2_norm = input_data.normalize(xtrain_s2,xtrain_s2)

    xtest_ns = input_data.normalize(xtrain,xtest)
    xtest_s1 = input_data.normalize(xtrain_s1,xtest)
    xtest_s2 = input_data.normalize(xtrain_s2,xtest)
    
    
    x_train_norm = np.empty(3,dtype = np.ndarray)
    x_train_norm[0],x_train_norm[1],x_train_norm[2] = xtrain_norm.values,xtrain_s1_norm,xtrain_s2_norm
    
    x_test_norm = np.empty(3,dtype = np.ndarray)
    x_test_norm[0],x_test_norm[1],x_test_norm[2] = xtest_ns.values,xtest_s1.values,xtest_s2.values
    
    y_train = np.empty(3,dtype = np.ndarray)
    y_train[0],y_train[1],y_train[2] = ytrain.values,ytrain_s1,ytrain_s2
    
    y_test = np.empty(3,dtype = np.ndarray)
    y_test[0],y_test[1],y_test[2] = ytest.values,ytest.values,ytest.values
    
    np.savez(save_path,xtrain = x_train_norm,xtest = x_test_norm,ytrain = y_train,ytest = y_test)
    
    del  xtrain,ytrain,xtrain_s1,ytrain_s1,xtrain_s2,ytrain_s2,x_test_norm,x_train_norm,xtest,xtest_ns
    del xtest_s1,xtest_s2,xtrain_norm,xtrain_s1_norm,xtrain_s2_norm,y_test,y_train,ytest

In [10]:
data = np.load(r'result/data_norm.npz',allow_pickle=True)

xtrain = data['xtrain']
xtest = data['xtest']
ytrain = data['ytrain']
ytest = data['ytest']

del data

# Data Summary

In [11]:
save_path = r'result/抽样数量.csv'
show = pd.DataFrame()
show['x_train'] = [k for k in map(lambda x:len(x),xtrain)]
show['x_test'] = [k for k in map(lambda x:len(x),xtest)]
show['Ratio'] = ['1:73','1:4','1:1']
show['total'] = show['x_train'] + show['x_test']

show.index = ['Origin','SMOTE1','SMOTE2']
show.to_csv(save_path)
show

,x_train,x_test,Ratio,total
Origin,62210,26662,1:73,88872
SMOTE1,76720,26662,1:4,103382
SMOTE2,122752,26662,1:1,149414


In [12]:
del xtrain_s1,xtrain_s2,ytrain_s1,ytrain_s2,show

# Greedy

In [15]:
save_path = r'result/greedy/greedy_index.npy'
if switch[3]:
    greedy_index = []
    for i,clf_i in enumerate(clf):
        if clf_switch[i]:
            temp = []
            for j in tqdm(range(len(xtrain))):
                choose = greedy.greedy(clf_i,xtrain[j],ytrain[j],xtest[j],ytest[j],[i for i in range(xtrain[0].shape[1])],8)
                temp.append(dcp(choose))
        else:
            temp = [None,None,None]
        greedy_index.append(temp)
    np.save(save_path,greedy_index)
    del greedy_index,temp,choose

In [16]:
greedy_index = np.load(save_path,allow_pickle=True)

# Greedy-Metrics

In [17]:
save_path = r'result/greedy/greedy_me.npy'
if switch[4]:
    greedy_me = []
    for i,clf_i in enumerate(clf):
        if clf_switch[i]:
            for j in tqdm(range(len(xtrain))):
                greedy_me.append(me.performance(clf_i,xtrain[j][:,greedy_index[i,j]],ytrain[j],xtest[j][:,greedy_index[i,j]],ytest[j]))
        else :
            for j in range(3):
                greedy_me.append([None,None,None,None,None,None,None,None])
    np.save(save_path,greedy_me)

  0%|                                                                                                                    | 0/3 [00:00<?, ?it/s]C:\Users\hasee\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
 33%|████████████████████████████████████                                                                        | 1/3 [00:05<00:10,  5.18s/it]C:\Users\hasee\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
 67%|████████████████████████████████████████████████████████████████████████                                    | 2/3 [00:13<00:06,  6.01s/it]C:\Users\hasee\An

epochs:  1 / 3
epochs:  2 / 3
epochs:  3 / 3


 33%|████████████████████████████████████                                                                        | 1/3 [00:07<00:14,  7.18s/it]

epochs:  1 / 3
70000 / 76720
epochs:  2 / 3
70000 / 76720
epochs:  3 / 3
70000 / 76720


 67%|████████████████████████████████████████████████████████████████████████                                    | 2/3 [00:17<00:07,  7.99s/it]

epochs:  1 / 3
70000 / 122752
epochs:  2 / 3
70000 / 122752
epochs:  3 / 3
70000 / 122752


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:34<00:00, 10.76s/it]


In [19]:
greedy_me = np.load(save_path,allow_pickle = True)

show = pd.DataFrame(index = pd.MultiIndex.from_product([clf_name,['Origin','SMOTE1','SMOTE2']]))
show['Feature_ID'] = np.reshape(greedy_index,(-1,1))[:,0]
show['Accuracy'] = greedy_me[:,2]
show['Recall'] = greedy_me[:,3]
show['Precision'] = greedy_me[:,4]
show['F1_Score'] = greedy_me[:,5]
show['FPR'] = greedy_me[:,6]
show.to_csv(r'result/性能度量1.csv')
show

Feature_ID  Accuracy    Recall Precision  \
DT        Origin             [1, 7, 4, 3, 8]  0.999512   0.97486  0.988669   
          SMOTE1             [4, 7, 1, 3, 9]   0.99955  0.988827  0.977901   
          SMOTE2            [4, 7, 1, 3, 11]  0.999475   0.99162  0.969945   
ANN       Origin  [4, 7, 3, 2, 5, 1, 10, 11]  0.999062  0.960894  0.969014   
          SMOTE1          [4, 7, 2, 8, 5, 6]  0.998912  0.994413  0.929504   
          SMOTE2                   [4, 6, 7]  0.995874  0.986034  0.770742   
Adaboost  Origin       [4, 6, 1, 2, 7, 3, 8]   0.99955  0.986034  0.980556   
          SMOTE1             [4, 7, 3, 2, 1]    0.9994  0.988827  0.967213   
          SMOTE2         [4, 7, 3, 2, 5, 10]  0.999175  0.988827  0.951613   
Xgboost   Origin            [4, 7, 1, 3, 11]  0.999625  0.988827  0.983333   
          SMOTE1          [4, 7, 1, 2, 6, 9]   0.99955  0.988827  0.977901   
          SMOTE2                [4, 7, 1, 3]  0.999437  0.988827  0.969863   
Gboosting Origin                [1, 7, 4, 5]  0.999362  0.977654   0.97493   
          SMOTE1            [4, 7, 2, 10, 8]   0.99955  0.988827  0.977901   
          SMOTE2             [4, 7, 1, 2, 3]  0.999325  0.988827  0.961957   
NB        Origin                   [4, 3, 2]  0.997712  0.980447  0.866667   
          SMOTE1                      [4, 3]    0.9976  0.972067  0.865672   
          SMOTE2                   [4, 3, 6]  0.997487  0.980447  0.854015   
VFDT      Origin                   [4, 6, 5]  0.997787  0.902235  0.930836   
          SMOTE1                   [4, 7, 5]  0.997337  0.938547  0.872727   
          SMOTE2                   [4, 5, 6]  0.996924  0.946927  0.843284   

                  F1_Score          FPR  
DT        Origin  0.981716  0.000152068  
          SMOTE1  0.983333  0.000304136  
          SMOTE2  0.980663  0.000418187  
ANN       Origin  0.964937  0.000418187  
          SMOTE1  0.960864   0.00102646  
          SMOTE2  0.865196   0.00399179  
Adaboost  Origin  0.983287  0.000266119  
          SMOTE1  0.977901  0.000456204  
          SMOTE2  0.969863  0.000684307  
Xgboost   Origin  0.986072  0.000228102  
          SMOTE1  0.983333  0.000304136  
          SMOTE2  0.979253  0.000418187  
Gboosting Origin   0.97629  0.000342153  
          SMOTE1  0.983333  0.000304136  
          SMOTE2  0.975207  0.000532238  
NB        Origin  0.920052   0.00205292  
          SMOTE1  0.915789   0.00205292  
          SMOTE2  0.912874   0.00228102  
VFDT      Origin  0.916312  0.000912409  
          SMOTE1  0.904441   0.00186283  
          SMOTE2  0.892105   0.00239507

In [20]:
del show,greedy_index,greedy_me

# All-Metrics

In [47]:
save_path = r'result/greedy/all_me.npy'
if switch[4]:
    all_me = []
    for i,clf_i in enumerate(clf):
        if clf_switch[i]:
            for j in tqdm(range(len(xtrain))):
                all_me.append(me.performance(clf_i,xtrain[j],ytrain[j],xtest[j],ytest[j]))
        else :
            for j in range(3):
                all_me.append([None,None,None,None,None,None,None,None])
    np.save(save_path,all_me)

  0%|                                                                                                                    | 0/3 [00:00<?, ?it/s]

epochs:  1 / 3
epochs:  2 / 3
epochs:  3 / 3


 33%|████████████████████████████████████                                                                        | 1/3 [00:08<00:16,  8.36s/it]

epochs:  1 / 3
70000 / 76720
epochs:  2 / 3
70000 / 76720
epochs:  3 / 3
70000 / 76720


 67%|████████████████████████████████████████████████████████████████████████                                    | 2/3 [00:20<00:09,  9.50s/it]

epochs:  1 / 3
70000 / 122752
epochs:  2 / 3
70000 / 122752
epochs:  3 / 3
70000 / 122752


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:55<00:00, 17.29s/it]


In [23]:
all_me = np.load(save_path,allow_pickle = True)

show = pd.DataFrame(index = pd.MultiIndex.from_product([clf_name,['Origin','SMOTE1','SMOTE2']]))
show['Feature_ID'] = np.repeat(['all'],21)
show['Accuracy'] = all_me[:,2]
show['Recall'] = all_me[:,3]
show['Precision'] = all_me[:,4]
show['F1_Score'] = all_me[:,5]
show['FPR'] = all_me[:,6]
show.to_csv(r'result/性能度量2.csv')
show

Feature_ID  Accuracy    Recall Precision  F1_Score  \
DT        Origin        all  0.999362  0.969274  0.983003   0.97609   
          SMOTE1        all    0.9994  0.977654  0.977654  0.977654   
          SMOTE2        all    0.9994  0.977654  0.977654  0.977654   
ANN       Origin        all  0.999175  0.960894  0.977273  0.969014   
          SMOTE1        all  0.999025  0.988827  0.941489  0.964578   
          SMOTE2        all  0.998687   0.98324  0.923885  0.952639   
Adaboost  Origin        all  0.999475  0.980447  0.980447  0.980447   
          SMOTE1        all   0.99925  0.986034  0.959239  0.972452   
          SMOTE2        all    0.9991  0.988827  0.946524  0.967213   
Xgboost   Origin        all  0.999512  0.977654  0.985915  0.981767   
          SMOTE1        all  0.999512  0.988827  0.975207  0.981969   
          SMOTE2        all  0.999475  0.988827  0.972527  0.980609   
Gboosting Origin        all  0.999475  0.977654  0.983146  0.980392   
          SMOTE1        all  0.999512  0.986034  0.977839  0.981919   
          SMOTE2        all  0.999362  0.986034  0.967123  0.976487   
NB        Origin        all  0.994974  0.994413  0.729508  0.841608   
          SMOTE1        all  0.994712  0.994413  0.719192  0.834701   
          SMOTE2        all  0.994036  0.997207  0.693204  0.817869   
VFDT      Origin        all    0.9976   0.97486  0.863861   0.91601   
          SMOTE1        all  0.986348         0         0         0   
          SMOTE2        all   0.98631         0         0         0   

                          FPR  
DT        Origin  0.000228102  
          SMOTE1  0.000304136  
          SMOTE2  0.000304136  
ANN       Origin  0.000304136  
          SMOTE1  0.000836375  
          SMOTE2   0.00110249  
Adaboost  Origin  0.000266119  
          SMOTE1  0.000570255  
          SMOTE2  0.000760341  
Xgboost   Origin  0.000190085  
          SMOTE1  0.000342153  
          SMOTE2   0.00038017  
Gboosting Origin  0.000228102  
          SMOTE1  0.000304136  
          SMOTE2  0.000456204  
NB        Origin   0.00501825  
          SMOTE1   0.00528437  
          SMOTE2   0.00600669  
VFDT      Origin   0.00209094  
          SMOTE1  0.000228102  
          SMOTE2  0.000266119